<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Manipulating-models" data-toc-modified-id="Manipulating-models-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Manipulating models</a></span><ul class="toc-item"><li><span><a href="#Making-temporary-changes-to-the-model" data-toc-modified-id="Making-temporary-changes-to-the-model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Making temporary changes to the model</a></span></li><li><span><a href="#Slim-versus-full-optimize" data-toc-modified-id="Slim-versus-full-optimize-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Slim versus full optimize</a></span></li><li><span><a href="#Changing-the-medium" data-toc-modified-id="Changing-the-medium-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Changing the medium</a></span></li><li><span><a href="#Exercise-(10-min)" data-toc-modified-id="Exercise-(10-min)-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Exercise (10 min)</a></span></li><li><span><a href="#Adding-reactions-and-pathways" data-toc-modified-id="Adding-reactions-and-pathways-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Adding reactions and pathways</a></span></li><li><span><a href="#Exercise-(20-min)" data-toc-modified-id="Exercise-(20-min)-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Exercise (20 min)</a></span></li></ul></li></ul></div>

# Manipulating models

In [ ]:
from cobra.io import read_sbml_model
model = read_sbml_model('data/e_coli_core.xml.gz')

## Making temporary changes to the model

Usually one relies on making copies if objects need to be changed but the original state needs to be retained. Unfortunately, making copies of models is time consuming.

In [ ]:
model.optimize()

In [ ]:
%%time
copy_of_model = model.copy()

Yes, even milliseconds add up pretty quickly if you need to run many simulation (e.g. if you need to knock out every single gene individually in the model to check if it is essential or not).

In [ ]:
%%time
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

For that reason cobrapy provides a mechanism that is less time consuming. Almost all methods that make changes to the model such as knocking-out genes, reactions, adding or removing metabolites, reactions etc can be automatically reverted upon exit from a python context. How this works is probably best understood by looking at an example.

In [ ]:
%%time
for gene in model.genes:
     with model:
            gene.knock_out()

Here, the `with model` statements starts the context and changes done to the model one indentation level to the right, are automatically recorded. When that block finishes, the context manager is requested to roll-back all changes leaving the model looking exactly as it did before all the changes.

Changing flux bounds can as indicated also be done reversibly. For example let's set the lower and upper bound of phosphoglycerate kinase to 0 (effectively knocking out the reaction).

In [ ]:
with model:
    model.reactions.PGK.bounds = 0, 0
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.bounds)

## Slim versus full optimize

Mathematical solvers are now so fast that for many small to mid-size models computing the solution can be even faster than it takes us to collect the values from the solver and convert that to objects that are usable for in python. When we use `model.optimize` we gather values for all reactions and metabolites and that can take some time. If we are only interested in the flux value of a single reaction or the objective, it is faster to instead use `model.slim_optimize` which only does the optimization and returns the objective value leaving it up to you to fetch other values that you may need. For example, let's optimize and get the flux value of the `ATPM` reaction.

In [ ]:
%%time
solution = model.optimize()
solution.fluxes['ATPM']

In [ ]:
%%time
model.slim_optimize()
model.reactions.ATPM.flux

Again, the difference may seem small but when done thousands of times these small differences can start to become significant.

## Changing the medium

One can access the medium condition using `model.medium`. The indicated bound is the effective upper uptake bound. 

In [ ]:
model.medium

Changing the carbon source in the medium can be achieved by adjusting the flux bounds of the respective exchange reactions appropriately. For example, the following code block removes glucose from the medium and adds succinate.

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 0
    medium['EX_succ_e'] = 10
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])

Changing the carbon source to succinate led to a significant drop in growth rate.

## Exercise (10 min)

* Change the carbon source in the medium to a different carbon source. What is the difference in the growth rate observed?
* How about growing E. coli under anaerobic conditions?

## Adding reactions and pathways

In [ ]:
from cobra import Reaction, Metabolite

Ok, let's create a new reactions.

In [ ]:
new_reaction = Reaction('alchemy')

This reaction is going to convert water into gold (unfortunately lead is not part of _E. coli _ metabolism; creating wine would be blasphemy). So we need to create a new metabolite, since gold is not yet part of _E. coli's_ native metabolism.

In [ ]:
gold = Metabolite(id='gold_c', compartment='c')

Now, we're going to specify the reaction's stoichiometry.

In [ ]:
new_reaction.add_metabolites({model.metabolites.h2o_c: -1, gold: 1})

Printing the reaction reveals that the reaction indeed converts water into gold.

In [ ]:
print(new_reaction.build_reaction_string())

Now, let's add the new reaction to the model.

In [ ]:
model.add_reactions([new_reaction])

Quickly check that the reaction was indeed added to the model.

In [ ]:
model.reactions.alchemy

Let's produce some gold then!

In [ ]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

:-(

What happened? Forgot to add an exchange reaction so that gold can leave the system.

In [ ]:
model.add_boundary(model.metabolites.gold_c, type='demand')

In [ ]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

Yes, much better!

## Exercise (20 min)

* Add a pathway to the model (ideally one that you're personally interested in; you can also use a different model if you like). 